# **Exploratory Data Analysis Notebook**

## Objectives

* Inspect and understand the dataset via Exploratory Data Analysis. Check Data Types, Missing Data, Variables and Correlations. Perform statistic analysis to gain insight into data.

## Inputs

* Android_Malware.csv

## Outputs

* EDA_Report.html
* Android_Malware_converted.csv

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

In [ ]:
import os
from pathlib import Path

# Resolve the project root
project_root = Path.cwd()
if project_root.name == "jupyter_notebooks":
    project_root = project_root.parent

# Import Libraries

In this section, all necessary standard libaries are imported to allow using their functions.

Import Libraries with necessary Settings

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
%matplotlib inline
sns.set(style="whitegrid")

# Load Raw Dataset

In this section, the raw dataset is loaded to be able to access the data.

Load Dataset from Inputs

In [ ]:
df = pd.read_csv(Path.cwd().parent / "inputs" / "datasets" / "raw" / "Android_Malware.csv")

---

# General Data Exploration

Get general overview of dataset

In [ ]:
df.info()

* The first inspection shows, that there are some columns that have an object data type. For further analysis, this needs to be converted to numeric.

* The expected target variable - Label - is also an object. This needs to be converted as well. 

---

# Pandas Profiling Report

In this section, a pandas profiling report is created via ydata_profiling. The report serves as a general overview for the whole dataset and is saved in outputs folder for future reference.

Import ydata_profiling and create and save Pandas Profile Report

In [ ]:
# Import Library
from ydata_profiling import ProfileReport

# Create Profiling Report
profile = ProfileReport(df=df, minimal=True)
profile.to_notebook_iframe()

# Save Report in Outputs Folder (create if not existing)
os.makedirs("outputs/eda", exist_ok=True)
profile.to_file(Path.cwd().parent / "outputs" / "eda" / "EDA_Report.html")

* The report shows detailed information about all variables. It allows to clearly identify the target variable - the label column.

* The label column has four different values: android_sms_malware, android_adware, android_scareware and benign. These describe the class / category of each sample in the dataset. 

* Project Goal: Create a classification model to predict the label based on the features.

* The other columns are potential input features that can be used for training the model. 

* Further study of missing values, data types and correlation is needed to see which features are useful for a prediction model for the label target variable.

---

# Data Preparation

In this section, 

Check all columns to see naming and formatting issues

In [ ]:
df.columns.tolist()

Clean column names to avoid hidden spaces

In [ ]:
df.columns = df.columns.str.strip()
df.columns.tolist()

Encode target variable Label and remove after encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Label_encoded'] = le.fit_transform(df['Label'])

# Print label mapping
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("🎯 Label mapping (target encoding):")
print(label_mapping)

# Drop original Label column after encoding
df.drop(columns=['Label'], inplace=True)
print("🗑️ Dropped original 'Label' column after encoding.")

Convert malformed numeric object columns to numeric & show results of conversion

In [ ]:
# Identify original object columns
original_obj_cols = df.select_dtypes(include='object').columns.tolist()
converted_cols = []

for col in original_obj_cols:
    try:
        df[col] = pd.to_numeric(df[col])
        converted_cols.append(col)
    except ValueError:
        pass  # Leave it as object if conversion fails

# Show conversion results
remaining_obj_cols = df.select_dtypes(include='object').columns.tolist()
print("\n✅ Successfully converted to numeric:")
print(converted_cols)

print("\n❌ Still categorical or string (object):")
print(remaining_obj_cols)

Check column values for suspicious looking numeric columns

In [ ]:
# Get only relevant columns
suspected_numeric = ['CWE Flag Count', 'Down/Up Ratio', 'Fwd Avg Bytes/Bulk']

# Preview unique values to spot issues
for col in suspected_numeric:
    print(f"\n🔍 Unique values in '{col}':")
    print(df[col].unique()[:10])

Convert unique values of object columns to NaN / numeric

In [ ]:
cols_to_clean = ['CWE Flag Count', 'Down/Up Ratio', 'Fwd Avg Bytes/Bulk']

# Convert to numeric (coerce errors to NaN)
for col in cols_to_clean:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\n✅ Finished converting suspicious numeric columns.")

Drop unhelpful metadata and index object columns

In [ ]:
columns_to_drop = ['Flow ID', 'Source IP', 'Destination IP', 'Timestamp', 'Unnamed: 0']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

print("\n🗑️ Dropped unhelpful metadata and index columns.")

Final check for remaining object columns

In [ ]:
final_obj_cols = df.select_dtypes(include='object').columns.tolist()
print("\n Final object columns (categorical candidates):")
print(final_obj_cols)

Save converted dataframe as a copy to keep original one intact & save as csv file for easier access

In [ ]:
# Make copy of dataframe for further use and easier separation
df_converted = df.copy()

# Save converted dataframe as csv file for easier later access of converted data
os.makedirs("outputs/data", exist_ok=True)
df_converted.to_csv(Path.cwd().parent / "outputs" / "data" / "Android_Malware_converted.csv", index=False)

print("✅ Saved converted dataframe as copy of original")
print("✅ Saved converted dataframe to outputs/data/")

---

# Load Converted Dataset

In this section, the converted dataset is loaded for further analysis. It is set to df_converted to keep code clear and understandable. 

Load converted dataset from outputs

In [ ]:
df_converted = pd.read_csv(Path.cwd().parent / "outputs" / "data" / "Android_Malware_converted.csv")

# Correlation Study

In this section,

Run Spearman and Pearson's correlation with target variable Label_encoded and print top 10 correlations for analysis

In [ ]:
# Spearman Correlation
corr_spearman = df_converted.corr(method='spearman')['Label_encoded'].sort_values(key=abs, ascending=False)[1:11]
print("📊 Top 10 Spearman correlations with Label_encoded:")
print(corr_spearman)

# Pearson Correlation
corr_pearson = df_converted.corr(method='pearson')['Label_encoded'].sort_values(key=abs, ascending=False)[1:11]
print("📊 Top 10 Pearson correlations with Label_encoded:")
print(corr_pearson)

Select top variables to explore further

In [ ]:
top_n = 5
vars_to_explore = list(set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list()))
print("🔎 Variables to explore further based on correlation:")
print(vars_to_explore)

---

# Explore Distributions by Target

In this section, 

In [ ]:
def plot_numerical(df, col, target_var):
    plt.figure(figsize=(8, 5))
    sns.histplot(data=df, x=col, hue=target_var, kde=True, element="step")
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()

target_var = 'Label_encoded'

# Subset dataframe for EDA
df_eda = df_converted[vars_to_explore + [target_var]]

# Plot each variable
for col in vars_to_explore:
        plot_numerical(df_eda, col, target_var)

---

# Explore Data Quality

In this section, 

Check for missing values per column

In [ ]:
missing = df_converted.isnull().sum()
print("Missing values per column:\n", missing[missing > 0] if missing.any() else "No missing values!")

Check for Outliers using IQR Method

In [ ]:
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return outliers.count()

print("\nOutliers detected per numeric column (IQR method):")
numeric_cols = df_converted.select_dtypes(include=['number']).columns
outliers_count = {col: detect_outliers_iqr(df_converted[col]) for col in numeric_cols}
for col, count in outliers_count.items():
    print(f"{col}: {count} outliers")

Check Skewness for numeric features

In [ ]:
skewness = df_converted[numeric_cols].skew()
print("\nSkewness per numeric column:")
print(skewness)

Check Distribution for selected numeric features

In [ ]:
def plot_distributions_fast(df, cols, n_cols=3):
    plt.figure(figsize=(15, 5 * ((len(cols) + n_cols - 1) // n_cols)))
    for i, col in enumerate(cols, 1):
        plt.subplot((len(cols) + n_cols - 1) // n_cols, n_cols, i)
        sns.histplot(df[col], bins=30, kde=False)  # KDE disabled and bins limited
        plt.title(f'Distribution of {col}')
    plt.tight_layout()
    plt.show()

# Plot only top 5 numeric features to save time
plot_distributions_fast(df_converted, numeric_cols[:5])

Check Class Balance

In [ ]:
print("\nClass distribution (target variable):")
print(df_converted['Label_encoded'].value_counts(normalize=True))
sns.countplot(x='Label_encoded', data=df_converted)
plt.title('Class Distribution')
plt.show()

* The class distribution shows that the dataset is imbalanced with the Benign (3) class being underrepresented. 

* This is important to keep in mind and might need adjusting later to avoid affecting the model performance negatively and prevent correctly identifying the benign class.

---

# Updated Pandas Profiling Report

In this section, an updated version of the Pandas Profiling report is created to allow for comparison between reports after preparing and changing the dataframe.

Import ydata_profiling and create and save Pandas Profile Report

In [ ]:
# Import Library
from ydata_profiling import ProfileReport

# Create Profiling Report
profile = ProfileReport(df=df_converted, minimal=True)
profile.to_notebook_iframe()

# Save Report in Outputs Folder (create if not existing)
os.makedirs("outputs/eda", exist_ok=True)
profile.to_file(Path.cwd().parent / "outputs" / "eda" / "EDA_Report_converted.html")

---

# Conclusion and Next Steps

* 